In [1]:
# 필수 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
from sklearn.preprocessing import StandardScaler

import gc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

data_root = '/content/drive/MyDrive/Colab Notebooks/final_project/data/'
# data_path = data_root + 'train/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# pyarrow 설치 (parquet 파일 읽기용)
!pip install pyarrow

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

In [4]:
# 데이터 로드: 6개월 청구정보 parquet 파일 병합
df41 = pd.read_parquet(data_root + 'test/4.청구입금정보/201807_test_청구정보.parquet')
df42 = pd.read_parquet(data_root + 'test/4.청구입금정보/201808_test_청구정보.parquet')
df43 = pd.read_parquet(data_root + 'test/4.청구입금정보/201809_test_청구정보.parquet')
df44 = pd.read_parquet(data_root + 'test/4.청구입금정보/201810_test_청구정보.parquet')
df45 = pd.read_parquet(data_root + 'test/4.청구입금정보/201811_test_청구정보.parquet')
df46 = pd.read_parquet(data_root + 'test/4.청구입금정보/201812_test_청구정보.parquet')

In [5]:
df4 = pd.concat([df41, df42, df43, df44, df45, df46])
print("병합 완료: ", df4.shape)

병합 완료:  (600000, 46)


In [6]:
# 불필요한 컬럼 제거
cols_to_drop = ['대표결제방법코드', '청구서발송여부_R3M', '청구금액_R3M', '포인트_마일리지_환산_B0M']
df4.drop(columns=cols_to_drop, inplace=True, errors='ignore')
print("컬럼 제거 완료")

컬럼 제거 완료


In [7]:
# '대표청구지고객주소구분코드' 컬럼 제거
df4.drop(columns=['대표청구지고객주소구분코드'], inplace=True, errors='ignore')

In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

# 수치형 컬럼만 선택
numeric_cols = df4.select_dtypes(include=['float64', 'int64']).columns.tolist()
X_vif = df4[numeric_cols].dropna()

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

# VIF가 10 이상인 피처 확인
high_vif_features = vif_data[vif_data["VIF"] > 10]["Feature"].tolist()
print("VIF > 10:", high_vif_features)

VIF > 10: ['기준년월', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_적립포인트_R3M', '포인트_이용포인트_R3M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '할인금액_R3M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '혜택수혜금액_R3M', '혜택수혜금액']


In [11]:
# 1. 기준년월 컬럼을 제외한 VIF 10 이상 컬럼만 제거 대상에 포함
features_to_drop = [col for col in high_vif_features if col != '기준년월']

# 2. 컬럼 제거
df4 = df4.drop(columns=features_to_drop)
print("제거된 컬럼 목록:", features_to_drop)

제거된 컬럼 목록: ['포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_적립포인트_R3M', '포인트_이용포인트_R3M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '할인금액_R3M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '혜택수혜금액_R3M', '혜택수혜금액']


In [12]:
# 기준년월이 여전히 남아있는지 확인
print("'기준년월' in df4.columns:", '기준년월' in df4.columns)

'기준년월' in df4.columns: True


In [13]:
from sklearn.preprocessing import LabelEncoder

# 범주형 컬럼 확인 및 인코딩
categorical_cols = df4.select_dtypes(include=['object', 'category']).columns

# Label Encoding 수행
for col in categorical_cols:
    le = LabelEncoder()
    df4[col] = le.fit_transform(df4[col].astype(str)) # 결측치 등을 문자열 'nan'으로 처리하여 인코딩

print("범주형 컬럼 인코딩 완료")

범주형 컬럼 인코딩 완료


In [14]:
# Parquet 저장
df4.to_parquet("test_4번eda_encoded.parquet", index=False)

# 파일 다운로드
from google.colab import files
files.download("test_4번eda_encoded.parquet")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>